# 🌡️ Oximetria e Capnografia na UTI

A monitorização contínua de oxigenação e ventilação é essencial para prevenir eventos sentinela. Este notebook reúne fundamentos, algoritmos práticos e miniaplicações para interpretar curvas e integrar dados ao manejo diário.

## 🎯 Objetivos
- Comparar tecnologias de oximetria (pulso, regional, venosa mista) e limitações.
- Interpretar curvas de capnografia nas fases I-IV e reconhecer padrões críticos.
- Utilizar correlações SpO₂/PaO₂ e gradiente alvéolo-arterial para tomada de decisão.
- Integrar alarmes e tendências a protocolos de sedação, ventilação e analgesia.

## 🧭 Algoritmo de Desaturação
```mermaid
graph TD
    A[SpO₂ < 92%] --> B{Curva capnográfica presente?}
    B -->|Não| C[Verificar deslocamento do sensor / parada de fluxo]
    B -->|Sim| D{EtCO₂ > 45 mmHg?}
    D -->|Sim| E[Hipoventilação → aumentar ventilação minuto]
    D -->|Não| F{EtCO₂ < 30 mmHg?}
    F -->|Sim| G[Hiperventilação ou baixo débito]
    F -->|Não| H[Avaliar FiO₂, complacência e secreções]
```

## 📈 Relação SpO₂ x PaO₂ (Estimativa)
Utilize a tabela a seguir como aproximação rápida em pacientes com curva de dissociação normal. Ajustes são necessários em casos de choque, anemia severa ou disfunção mitocondrial.

In [ ]:
import numpy as np
import pandas as pd

spo2 = np.arange(75, 101, 1)
paO2 = 100 * np.exp(-((100 - spo2) / 26))

aproximacao = pd.DataFrame({
    'SpO₂ (%)': spo2,
    'PaO₂ estimada (mmHg)': paO2.round(1),
    'Classificação': np.select([spo2 < 88, spo2 < 92, spo2 >= 92], ['Hipoxemia grave', 'Zona de alerta', 'Meta adequada'])
})

aproximacao.head(10)

### Aplicação prática
- **Hipoxemia grave**: acionar protocolo de via aérea difícil e checar tubo/endoscopia.
- **Zona de alerta**: reavaliar ventilação, otimizar PEEP ou recrutamento.
- **Meta adequada**: manter titulação de FiO₂ < 60% sempre que possível.

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=aproximacao['SpO₂ (%)'], y=aproximacao['PaO₂ estimada (mmHg)'], mode='lines', name='Curva estimada'))
fig.add_vrect(x0=88, x1=92, fillcolor='#fff3cd', opacity=0.4, line_width=0)
fig.add_vrect(x0=75, x1=88, fillcolor='#f8d7da', opacity=0.4, line_width=0)
fig.add_vrect(x0=92, x1=100, fillcolor='#d1e7dd', opacity=0.3, line_width=0)
fig.update_layout(title='Curva estimada SpO₂ vs PaO₂', xaxis_title='SpO₂ (%)', yaxis_title='PaO₂ (mmHg)', height=400)
fig.show()

## 🧮 Calculadora de Gradiente Alvéolo-Arterial
O gradiente A-a ajuda a diferenciar causas de hipoxemia. Informe os valores atuais para obter interpretação automatizada.

In [ ]:
from ipywidgets import interact, FloatSlider

R = 0.8

def gradiente_aa(fio2: float = 0.5, pao2: float = 60.0, paco2: float = 40.0, barometrica: float = 760.0):
    'Calcula gradiente A-a e sugere causas prováveis.'
    alveolar = fio2 * (barometrica - 47) - paco2 / R
    gradiente = alveolar - pao2

    if gradiente < 10:
        interpretacao = 'Gradiente normal: suspeite hipoventilação pura.'
    elif gradiente < 25:
        interpretacao = 'Leve aumento: investigar atelectasia ou início de shunt.'
    elif gradiente < 50:
        interpretacao = 'Moderado: provável shunt intrapulmonar ou V/Q.'
    else:
        interpretacao = 'Importante: considere SDRA, embolia ou shunt intracardíaco.'

    return {
        'Pressão alveolar de O₂ (mmHg)': round(alveolar, 1),
        'Gradiente A-a (mmHg)': round(gradiente, 1),
        'Interpretação': interpretacao,
    }


interact(
    gradiente_aa,
    fio2=FloatSlider(value=0.5, min=0.21, max=1.0, step=0.01, description='FiO₂'),
    pao2=FloatSlider(value=60, min=30, max=200, step=1, description='PaO₂'),
    paco2=FloatSlider(value=40, min=20, max=80, step=1, description='PaCO₂'),
    barometrica=FloatSlider(value=760, min=500, max=780, step=1, description='PB (mmHg)'),
)

## 📉 Tipos de Curvas Capnográficas
- **Obstrução expiratória**: fase III com inclinação aumentada (asma, DPOC).
- **Reinalação**: ausência de retorno à linha de base (CO₂ > 0 na fase I).
- **Hiperventilação**: EtCO₂ < 30 mmHg e redução global da curva.
- **Apneia/parada de fluxo**: linha reta próxima de zero.

Acompanhe mudanças após ajustes de sedação, broncodilatador ou recrutamento alveolar.

In [ ]:
import numpy as np
import plotly.graph_objects as go

t = np.linspace(0, 6, 500)
co2_normal = 40 + 5 * np.sin(0.5 * np.pi * t)
co2_obstrucao = 42 + 10 * np.sin(0.5 * np.pi * t) ** 1.5
co2_apneia = np.zeros_like(t)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=np.clip(co2_normal, 0, 50), name='Normal', mode='lines'))
fig.add_trace(go.Scatter(x=t, y=np.clip(co2_obstrucao, 0, 55), name='Obstrução', mode='lines'))
fig.add_trace(go.Scatter(x=t, y=co2_apneia, name='Apneia', mode='lines'))
fig.update_layout(title='Padrões simulados de capnografia', xaxis_title='Tempo (s)', yaxis_title='EtCO₂ (mmHg)', height=400)
fig.show()

## 🧠 Caso Clínico Integrado
**Situação:** Paciente com SDRA em ventilação controlada, FiO₂ 0,6, PEEP 12 cmH₂O. SpO₂ cai para 86% enquanto EtCO₂ permanece 32 mmHg com curva normal.

1. Verifique posicionamento do tubo e presença de secreções grossas.
2. Execute recrutamento alveolar e ajuste PEEP se complacência permitir.
3. Reavalie após 5 minutos com nova gasometria e curva P/V do respirador.

**Discussão:** Queda isolada de SpO₂ com EtCO₂ estável sugere piora na relação V/Q ou shunt, não hipoventilação. Foque em intervenções pulmonares e hemodinâmicas.

## 📚 Referências
- Capnography. *Clinical application of capnography in mechanically ventilated patients*. Crit Care, 2020.
- O'Driscoll, B. et al. *British Thoracic Society Guideline for oxygen use in adults in healthcare*. BMJ Open Resp Res, 2017.
- Surviving Sepsis Campaign 2021 – recomendações de monitorização.
- Miller's Anesthesia, capítulo sobre monitorização respiratória.